# Preparation

First things first... Let's read the parquet file and take a look at what's inside

In [1]:
# reading the CSVs
import pandas as pd
import numpy as np

df = pd.read_parquet('../data/dataset.gz.parquet')
policy_data = df.copy() # to keep raw data untouched

In [2]:
policy_data.head()

,policy_holder_zipcode,policy_id,policy_start_date,policy_exposure_days,policy_claims_num_reported,policy_claims_num_paid,policy_claims_total_amount_paid_brl,policy_premium_received_brl,policy_holder_birth_date,policy_holder_gender,policy_holder_bonus_clas,policy_holder_residence_latitude,policy_holder_residence_longitude,vehicle_brand,vehicle_model,vehicle_make_year,vehicle_tarif_class,vehicle_value_brl,policy_holder_residence_region,policy_holder_residence_city
0,1000.0,22036576975396,20171229,58,0.0,0.0,0.0,777.432433,19420305,M,8.0,-23.513405,-46.535469,Renault,LOGAN Expression Hi-Flex 1.6 8V 4p,2017.0,Passeio Nacional,41652.0,METROPOLITANA DE SÃO PAULO,SÃO PAULO
1,1001.0,01472343000,20180530,214,0.0,0.0,0.0,619.134247,19850617,M,0.0,-23.547810,-46.632953,GM - Chevrolet,Celta Life/ LS 1.0 MPFI 8V FlexPower 5p,2007.0,Passeio Nacional,13138.0,METROPOLITANA DE SÃO PAULO,SÃO PAULO
2,1001.0,08024270000,20180307,298,0.0,0.0,0.0,1295.687671,19891230,M,1.0,-23.547810,-46.632953,Toyota,ETIOS XS Sedan1.5 Flex 16V 4p Mec.,2018.0,Passeio Nacional,47745.0,METROPOLITANA DE SÃO PAULO,SÃO PAULO
3,1003.0,166871902448270,20180428,246,0.0,0.0,0.0,2971.898688,19770307,M,4.0,-23.548744,-46.635580,Fiat,Palio 1.0/ Trofeo 1.0 Fire/ Fire Flex 4p,2015.0,Passeio Nacional,26165.0,METROPOLITANA DE SÃO PAULO,SÃO PAULO
4,1004.0,13975805000,20170918,259,0.0,0.0,0.0,1847.060274,19880204,M,3.0,-23.549011,-46.635594,Toyota,Corolla XEi 2.0 Flex 16V Aut.,2014.0,Passeio Nacional,58020.0,METROPOLITANA DE SÃO PAULO,SÃO PAULO


In [3]:
policy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3024172 entries, 0 to 3024171
Data columns (total 20 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   policy_holder_zipcode                float64
 1   policy_id                            object 
 2   policy_start_date                    object 
 3   policy_exposure_days                 int32  
 4   policy_claims_num_reported           float64
 5   policy_claims_num_paid               float64
 6   policy_claims_total_amount_paid_brl  float64
 7   policy_premium_received_brl          float64
 8   policy_holder_birth_date             int32  
 9   policy_holder_gender                 object 
 10  policy_holder_bonus_clas             float64
 11  policy_holder_residence_latitude     float64
 12  policy_holder_residence_longitude    float64
 13  vehicle_brand                        object 
 14  vehicle_model                        object 
 15  vehicle_make_year               

Not sure why, but the df.info() command didn't show me the count of NULL values. So here it is:

In [4]:
policy_data.isnull().sum()

policy_holder_zipcode                       0
policy_id                                   0
policy_start_date                           0
policy_exposure_days                        0
policy_claims_num_reported                  0
policy_claims_num_paid                      0
policy_claims_total_amount_paid_brl         0
policy_premium_received_brl                 0
policy_holder_birth_date                    0
policy_holder_gender                        0
policy_holder_bonus_clas               277313
policy_holder_residence_latitude            0
policy_holder_residence_longitude           0
vehicle_brand                               0
vehicle_model                               0
vehicle_make_year                           0
vehicle_tarif_class                         0
vehicle_value_brl                           0
policy_holder_residence_region              0
policy_holder_residence_city                0
dtype: int64

There are a few fields that I want to convert to better data types.

In [5]:
# weird way to check if I can safely convert to int the fields I want
# extracting the decimal places and validating if they're all 0s
print(policy_data["policy_holder_zipcode"].apply(lambda x: abs(x % 1)).sum())
print(policy_data["policy_claims_num_reported"].apply(lambda x: abs(x % 1)).sum())
print(policy_data["policy_claims_num_paid"].apply(lambda x: abs(x % 1)).sum())
print(policy_data["policy_holder_bonus_clas"].apply(lambda x: abs(x % 1)).sum())
print(policy_data["vehicle_make_year"].apply(lambda x: abs(x % 1)).sum())


0.0
0.0
0.0
0.0
0.0


In [6]:
# converting to timestamps
policy_data["policy_start_date"] = pd.to_datetime(policy_data["policy_start_date"])
policy_data["policy_holder_birth_date"] = pd.to_datetime(policy_data["policy_holder_birth_date"].astype(str))
policy_data["vehicle_make_year"] = pd.to_datetime(policy_data["vehicle_make_year"].astype(int), format='%Y')

# converting to string
policy_data["policy_holder_zipcode"] = policy_data["policy_holder_zipcode"].astype(int).astype(str)
policy_data["policy_holder_bonus_clas"] = policy_data["policy_holder_bonus_clas"].astype('Int64').astype(str) # using Int64 to be able to handle NULLs

# converting to int
policy_data["policy_claims_num_reported"] = policy_data["policy_claims_num_reported"].astype(int)
policy_data["policy_claims_num_paid"] = policy_data["policy_claims_num_paid"].astype(int)

# reordering columns
policy_data = policy_data[["policy_id",
                           "policy_start_date",
                           "policy_exposure_days",
                           "policy_premium_received_brl",
                           "policy_claims_num_reported",
                           "policy_claims_num_paid",
                           "policy_claims_total_amount_paid_brl",
                           "policy_holder_birth_date",
                           "policy_holder_gender",
                           "policy_holder_residence_city",
                           "policy_holder_residence_region",
                           "policy_holder_zipcode",
                           "policy_holder_residence_latitude",
                           "policy_holder_residence_longitude",
                           "policy_holder_bonus_clas",
                           "vehicle_brand",
                           "vehicle_model",
                           "vehicle_make_year",
                           "vehicle_tarif_class",
                           "vehicle_value_brl"
                         ]]

policy_data.head()

,policy_id,policy_start_date,policy_exposure_days,policy_premium_received_brl,policy_claims_num_reported,policy_claims_num_paid,policy_claims_total_amount_paid_brl,policy_holder_birth_date,policy_holder_gender,policy_holder_residence_city,policy_holder_residence_region,policy_holder_zipcode,policy_holder_residence_latitude,policy_holder_residence_longitude,policy_holder_bonus_clas,vehicle_brand,vehicle_model,vehicle_make_year,vehicle_tarif_class,vehicle_value_brl
0,22036576975396,2017-12-29,58,777.432433,0,0,0.0,1942-03-05,M,SÃO PAULO,METROPOLITANA DE SÃO PAULO,1000,-23.513405,-46.535469,8,Renault,LOGAN Expression Hi-Flex 1.6 8V 4p,2017-01-01,Passeio Nacional,41652.0
1,01472343000,2018-05-30,214,619.134247,0,0,0.0,1985-06-17,M,SÃO PAULO,METROPOLITANA DE SÃO PAULO,1001,-23.547810,-46.632953,0,GM - Chevrolet,Celta Life/ LS 1.0 MPFI 8V FlexPower 5p,2007-01-01,Passeio Nacional,13138.0
2,08024270000,2018-03-07,298,1295.687671,0,0,0.0,1989-12-30,M,SÃO PAULO,METROPOLITANA DE SÃO PAULO,1001,-23.547810,-46.632953,1,Toyota,ETIOS XS Sedan1.5 Flex 16V 4p Mec.,2018-01-01,Passeio Nacional,47745.0
3,166871902448270,2018-04-28,246,2971.898688,0,0,0.0,1977-03-07,M,SÃO PAULO,METROPOLITANA DE SÃO PAULO,1003,-23.548744,-46.635580,4,Fiat,Palio 1.0/ Trofeo 1.0 Fire/ Fire Flex 4p,2015-01-01,Passeio Nacional,26165.0
4,13975805000,2017-09-18,259,1847.060274,0,0,0.0,1988-02-04,M,SÃO PAULO,METROPOLITANA DE SÃO PAULO,1004,-23.549011,-46.635594,3,Toyota,Corolla XEi 2.0 Flex 16V Aut.,2014-01-01,Passeio Nacional,58020.0


Much better... Not even sure if all those transformation were actually necessary, but it is always better to tell python the correct data types. That might be helpful later when creating the graphs. Also, reordering the columns helps me to think when looking at a data sample.

Now I feel ready for the analysis itself.

After reading the loss ratio concept and the proposed excercise, it is really important to notice that the average of individual loss ratios is different than the total loss ratio for a specific group.

In [7]:
# individual loss_ratio calculation
policy_data["policy_loss_ratio"] = policy_data["policy_claims_total_amount_paid_brl"]/policy_data["policy_premium_received_brl"]

Running some basic descriptive statistics on all columns to better understand the data:

In [8]:
# Dates
policy_data.describe(include=[np.datetime64], datetime_is_numeric=True)

,policy_start_date,policy_holder_birth_date,vehicle_make_year
count,3024172,3024172,3024172
mean,2018-01-13 20:11:56.892557568,1969-12-21 10:53:28.684162140,2012-04-02 04:38:36.534905600
min,2017-01-02 00:00:00,1900-01-01 00:00:00,1985-01-01 00:00:00
25%,2017-08-12 00:00:00,1960-01-13 00:00:00,2010-01-01 00:00:00
50%,2018-01-19 00:00:00,1972-01-12 00:00:00,2013-01-01 00:00:00
75%,2018-06-15 00:00:00,1981-10-31 00:00:00,2015-01-01 00:00:00
max,2018-12-30 00:00:00,2009-03-09 00:00:00,2018-01-01 00:00:00


In [9]:
# Possible dimensions for the market analysis
policy_data.describe(include=[object])

,policy_id,policy_holder_gender,policy_holder_residence_city,policy_holder_residence_region,policy_holder_zipcode,policy_holder_bonus_clas,vehicle_brand,vehicle_model,vehicle_tarif_class
count,3024172,3024172,3024172,3024172,3024172,3024172,3024172,3024172,3024172
unique,3024172,2,831,52,7544,10,52,3314,10
top,22036576975396,M,SÃO PAULO,METROPOLITANA DE SÃO PAULO,15104,0,GM - Chevrolet,HB20 Comf./C.Plus/C.Style 1.0 Flex 12V,Passeio Nacional
freq,1,1514773,830938,1171145,12012,569156,702968,57989,2703425


In [10]:
# Numerical values
policy_data.describe(include=[np.number])

,policy_exposure_days,policy_premium_received_brl,policy_claims_num_reported,policy_claims_num_paid,policy_claims_total_amount_paid_brl,policy_holder_residence_latitude,policy_holder_residence_longitude,vehicle_value_brl,policy_loss_ratio
count,3.024172e+06,3.024172e+06,3.024172e+06,3.024172e+06,3.024172e+06,3.024172e+06,3.024172e+06,3.024172e+06,3.024172e+06
mean,1.393440e+02,4.956020e+02,2.545788e-02,1.379816e-02,1.703693e+02,-2.244564e+01,-4.705274e+01,3.688626e+04,5.020848e-01
std,9.240343e+01,5.129012e+02,1.667383e-01,1.195834e-01,2.328326e+03,2.582536e+00,1.766081e+00,2.375967e+04,1.968787e+01
min,1.000000e+00,1.598174e-03,0.000000e+00,0.000000e+00,0.000000e+00,-3.243358e+01,-6.252345e+01,2.996000e+03,0.000000e+00
25%,6.100000e+01,1.670872e+02,0.000000e+00,0.000000e+00,0.000000e+00,-2.355558e+01,-4.759808e+01,2.246300e+04,0.000000e+00
50%,1.350000e+02,3.730192e+02,0.000000e+00,0.000000e+00,0.000000e+00,-2.317706e+01,-4.679697e+01,3.236900e+04,0.000000e+00
75%,1.830000e+02,6.603041e+02,0.000000e+00,0.000000e+00,0.000000e+00,-2.225076e+01,-4.654857e+01,4.347300e+04,0.000000e+00
max,3.640000e+02,4.726853e+04,5.000000e+00,5.000000e+00,4.730860e+05,3.147420e+00,-3.480740e+01,2.820535e+06,1.342005e+04


I noticed that there are some birth dates dated 1900 (a bit odd). Let me double check that:

In [11]:
policy_data["policy_holder_birth_year"] = pd.DatetimeIndex(policy_data["policy_holder_birth_date"]).year
elderly_sample = policy_data[policy_data["policy_holder_birth_year"] < 1930] # older than ~90
elderly_sample["policy_holder_birth_year"].value_counts()

1900    36748
1929      860
1928      618
1927      440
1926      314
1925      215
1924      168
1923       59
1922       42
1920       31
1921       17
1919        8
1918        3
1909        1
1917        1
1916        1
Name: policy_holder_birth_year, dtype: int64

Indeed a significant number of policies is being assigned (very likely incorrectly) birth dates on 1900. To be considered in case we do anything using birth dates.

In [12]:
loss_ratio_analysis = policy_data[["policy_id",
                                   "policy_premium_received_brl",
                                   "policy_claims_total_amount_paid_brl",
                                   "policy_loss_ratio",
                                   "policy_holder_birth_year",
                                   "policy_holder_gender",
                                   "policy_holder_residence_city",
                                   "policy_holder_residence_region",
                                   # "policy_holder_zipcode",
                                   # "policy_holder_residence_latitude",
                                   # "policy_holder_residence_longitude",
                                   "policy_holder_bonus_clas",
                                   "vehicle_brand",
                                   # "vehicle_model",
                                   # "vehicle_make_year",
                                   "vehicle_tarif_class",
                                 ]]

In [13]:
# overall loss_ratio
print(loss_ratio_analysis["policy_claims_total_amount_paid_brl"].sum()/loss_ratio_analysis["policy_premium_received_brl"].sum())

# loss_ratio by gender
grouped = loss_ratio_analysis.groupby(by=["policy_holder_gender"])
claims_paid = grouped["policy_claims_total_amount_paid_brl"].agg(["sum","count"])
premium_received = grouped["policy_premium_received_brl"].agg(["sum","count"])
gender_loss_ratio = pd.DataFrame({"loss_ratio": claims_paid["sum"]/premium_received["sum"], "policy_count": premium_received["count"]})

gender_loss_ratio.sort_values(by="loss_ratio")

0.343762287767103


,loss_ratio,policy_count
policy_holder_gender,,
F,0.335485,1509399
M,0.351023,1514773


In [14]:
# loss_ratio by region
grouped = loss_ratio_analysis.groupby(by=["policy_holder_residence_region"])
claims_paid = grouped["policy_claims_total_amount_paid_brl"].agg(["sum","count"])
premium_received = grouped["policy_premium_received_brl"].agg(["sum","count"])
region_loss_ratio = pd.DataFrame({"loss_ratio": claims_paid["sum"]/premium_received["sum"], "policy_count": premium_received["count"]})

region_loss_ratio[region_loss_ratio["policy_count"]>200].sort_values(by="loss_ratio")

,loss_ratio,policy_count
policy_holder_residence_region,,
"ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, DIADEMA",0.271126,175391
VALE DO RIBEIRA,0.294500,22223
METROPOLITANA DE SÃO PAULO,0.298284,1171145
BAIXADA SANTISTA,0.301878,32972
VALE DO PARAÍBA,0.339484,146224
LITORAL NORTE DE SÃO PAULO,0.355432,19618
CAMPINAS - CIDADE,0.364066,111630
SOROCABA - REGIÃO 2,0.388032,286221
"SÃO JOSÉ DOS CAMPOS, JACAREÍ E CAÇAPAVA",0.390653,78354


In [15]:
# loss_ratio by city
grouped = loss_ratio_analysis.groupby(by=["policy_holder_residence_region", "policy_holder_residence_city"])
claims_paid = grouped["policy_claims_total_amount_paid_brl"].agg(["sum","count"])
premium_received = grouped["policy_premium_received_brl"].agg(["sum","count"])
city_loss_ratio = pd.DataFrame({"loss_ratio": claims_paid["sum"]/premium_received["sum"], "policy_count": premium_received["count"]})

city_loss_ratio[city_loss_ratio["policy_count"]>20000].sort_values(by="loss_ratio")

loss_ratio  \
policy_holder_residence_region                     policy_holder_residence_city               
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... DIADEMA                         0.218144   
BAIXADA SANTISTA                                   PRAIA GRANDE                    0.241926   
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... SANTO ANDRÉ                     0.248020   
                                                   SÃO CAETANO DO SUL              0.254973   
METROPOLITANA DE SÃO PAULO                         FRANCISCO MORATO                0.264812   
VALE DO PARAÍBA                                    SUZANO                          0.265059   
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... MAUÁ                            0.265470   
SOROCABA - REGIÃO 2                                SALTO                           0.268131   
METROPOLITANA DE SÃO PAULO                         CARAPICUÍBA                     0.275971   
                                                   GUARULHOS                       0.285109   
DEMAIS CAMPINAS 1                                  SUMARÉ                          0.290910   
METROPOLITANA DE SÃO PAULO                         SÃO PAULO                       0.291184   
                                                   OSASCO                          0.296750   
BAIXADA SANTISTA                                   GUARUJÁ                         0.316857   
SOROCABA - REGIÃO 2                                RIO CLARO                       0.317469   
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... SÃO BERNARDO DO CAMPO           0.324643   
METROPOLITANA DE SÃO PAULO                         SANTOS                          0.327721   
VALE DO PARAÍBA                                    MOGI DAS CRUZES                 0.346613   
METROPOLITANA DE SÃO PAULO                         COTIA                           0.351926   
VALE DO PARAÍBA                                    TAUBATÉ                         0.358231   
SOROCABA - REGIÃO 2                                LIMEIRA                         0.358909   
CAMPINAS - CIDADE                                  CAMPINAS                        0.364066   
METROPOLITANA DE SÃO PAULO                         TABOÃO DA SERRA                 0.366545   
SÃO JOSÉ DOS CAMPOS, JACAREÍ E CAÇAPAVA            JACAREÍ                         0.367822   
SOROCABA - REGIÃO 2                                ITU                             0.367936   
DEMAIS INTERIOR DE SÃO PAULO                       MARÍLIA                         0.376176   
SOROCABA - CIDADE E REGIÃO                         SOROCABA                        0.379884   
                                                   PIRACICABA                      0.388847   
                                                   SANTA BARBARA D´OESTE           0.391652   
SÃO JOSÉ DOS CAMPOS, JACAREÍ E CAÇAPAVA            SÃO JOSÉ DOS CAMPOS             0.396622   
DEMAIS CAMPINAS 1                                  VALINHOS                        0.403474   
DEMAIS INTERIOR DE SÃO PAULO                       SERTÃOZINHO                     0.409024   
SOROCABA - REGIÃO 2                                ATIBAIA                         0.409917   
SOROCABA - CIDADE E REGIÃO                         ARARAS                          0.411267   
METROPOLITANA DE SÃO PAULO                         SANTANA DO PARNAÍBA             0.416255   
DEMAIS INTERIOR DE SÃO PAULO                       SÃO CARLOS                      0.416696   
SOROCABA - REGIÃO 2                                BOTUCATU                        0.417184   
DEMAIS CAMPINAS 1                                  JUNDIAÍ                         0.423737   
METROPOLITANA DE SÃO PAULO                         BARUERI                         0.433111   
SOROCABA - CIDADE E REGIÃO                         AMERICANA                       0.434413   
DEMAIS INTERIOR DE SÃO PAULO                       MATAO                           0.437230

In [16]:
# loss_ratio by bonus_clas
grouped = loss_ratio_analysis.groupby(by=["policy_holder_bonus_clas"])
claims_paid = grouped["policy_claims_total_amount_paid_brl"].agg(["sum","count"])
premium_received = grouped["policy_premium_received_brl"].agg(["sum","count"])
bonus_clas_loss_ratio = pd.DataFrame({"loss_ratio": claims_paid["sum"]/premium_received["sum"], "policy_count": premium_received["count"]})

bonus_clas_loss_ratio.sort_values(by="loss_ratio")

,loss_ratio,policy_count
policy_holder_bonus_clas,,
4,0.316034,309238
8,0.317195,246696
5,0.320635,301888
6,0.324383,274402
3,0.324830,317333
1,0.348932,343213
2,0.351345,318956
<NA>,0.356339,277313
0,0.384800,569156


In [17]:
# loss_ratio by vehicle_brand
grouped = loss_ratio_analysis.groupby(by=["vehicle_brand"])
claims_paid = grouped["policy_claims_total_amount_paid_brl"].agg(["sum","count"])
premium_received = grouped["policy_premium_received_brl"].agg(["sum","count"])
vehicle_brand_loss_ratio = pd.DataFrame({"loss_ratio": claims_paid["sum"]/premium_received["sum"], "policy_count": premium_received["count"]})

vehicle_brand_loss_ratio[vehicle_brand_loss_ratio["policy_count"]>500].sort_values(by="loss_ratio")

,loss_ratio,policy_count
vehicle_brand,,
LIFAN,0.156527,1790
Porsche,0.200447,967
Volvo,0.230154,2428
Fiat,0.279694,374685
VW - VolksWagen,0.298954,459760
MINI,0.305272,5470
Subaru,0.310842,1824
Ford,0.315429,345901
Hyundai,0.320025,186434


These give us some indication of market groups that we could investigate further. I'd need to refresh my memory and python skills, but I'm sure there are robust statistical techniques (ANOVA, K-Means clustering, etc) that could be used to better analyse the data.

We'd need to check the data's variance to actually identify what a "small loss_ratio" would be, but for now I could list the groups we identified (using arbritary min sample sizes) with loss_ratio less than 30%:

In [18]:
#region
small_region_loss_ratio = region_loss_ratio[region_loss_ratio["policy_count"]>200].sort_values(by="loss_ratio")
small_region_loss_ratio = small_region_loss_ratio[region_loss_ratio["loss_ratio"]<0.3]
small_region_loss_ratio

/var/folders/wf/z578m42550bgsqj4zdbtlxjw0000gn/T/ipykernel_11402/593868242.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  small_region_loss_ratio = small_region_loss_ratio[region_loss_ratio["loss_ratio"]<0.3]


,loss_ratio,policy_count
policy_holder_residence_region,,
"ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, DIADEMA",0.271126,175391
VALE DO RIBEIRA,0.294500,22223
METROPOLITANA DE SÃO PAULO,0.298284,1171145


In [19]:
#city
small_city_loss_ratio = city_loss_ratio[city_loss_ratio["policy_count"]>10000].sort_values(by="loss_ratio")
small_city_loss_ratio = small_city_loss_ratio[city_loss_ratio["loss_ratio"]<0.3]
small_city_loss_ratio

/var/folders/wf/z578m42550bgsqj4zdbtlxjw0000gn/T/ipykernel_11402/356288018.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  small_city_loss_ratio = small_city_loss_ratio[city_loss_ratio["loss_ratio"]<0.3]


loss_ratio  \
policy_holder_residence_region                     policy_holder_residence_city               
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... DIADEMA                         0.218144   
BAIXADA SANTISTA                                   PRAIA GRANDE                    0.241926   
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... SANTO ANDRÉ                     0.248020   
                                                   SÃO CAETANO DO SUL              0.254973   
METROPOLITANA DE SÃO PAULO                         FRANCISCO MORATO                0.264812   
VALE DO PARAÍBA                                    SUZANO                          0.265059   
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... MAUÁ                            0.265470   
SOROCABA - REGIÃO 2                                SALTO                           0.268131   
METROPOLITANA DE SÃO PAULO                         CARAPICUÍBA                     0.275971   
                                                   GUARULHOS                       0.285109   
DEMAIS CAMPINAS 1                                  SUMARÉ                          0.290910   
METROPOLITANA DE SÃO PAULO                         SÃO PAULO                       0.291184   
                                                   OSASCO                          0.296750   

                                                                                 policy_count  
policy_holder_residence_region                     policy_holder_residence_city                
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... DIADEMA                              15726  
BAIXADA SANTISTA                                   PRAIA GRANDE                         12435  
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... SANTO ANDRÉ                          58222  
                                                   SÃO CAETANO DO SUL                   32920  
METROPOLITANA DE SÃO PAULO                         FRANCISCO MORATO                     62674  
VALE DO PARAÍBA                                    SUZANO                               16741  
ABCD, SANTO ANDRÉ, SÃO BERNARDO, SÃO CAETANO, D... MAUÁ                                 13939  
SOROCABA - REGIÃO 2                                SALTO                                10699  
METROPOLITANA DE SÃO PAULO                         CARAPICUÍBA                          14757  
                                                   GUARULHOS                            69050  
DEMAIS CAMPINAS 1                                  SUMARÉ                               13652  
METROPOLITANA DE SÃO PAULO                         SÃO PAULO                           830938  
                                                   OSASCO                               37616

In [21]:
#vehicle_brand
small_vehicle_brand_loss_ratio = vehicle_brand_loss_ratio[vehicle_brand_loss_ratio["policy_count"]>500].sort_values(by="loss_ratio")
small_vehicle_brand_loss_ratio = small_vehicle_brand_loss_ratio[vehicle_brand_loss_ratio["loss_ratio"]<0.3]
small_vehicle_brand_loss_ratio

/var/folders/wf/z578m42550bgsqj4zdbtlxjw0000gn/T/ipykernel_11402/3055018983.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  small_vehicle_brand_loss_ratio = small_vehicle_brand_loss_ratio[vehicle_brand_loss_ratio["loss_ratio"]<0.3]


,loss_ratio,policy_count
vehicle_brand,,
LIFAN,0.156527,1790
Porsche,0.200447,967
Volvo,0.230154,2428
Fiat,0.279694,374685
VW - VolksWagen,0.298954,459760
